In [1]:
from datetime import datetime, timedelta
import uuid
import values, helpers
import random
import pandas as pd
import numpy as np
import create_sap_table.create_table_leanx as sap_table

In [11]:
# required tables
VBAK = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='VBAK')])
VBAP = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='VBAP')])

LIKP = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='LIKP')])
LIPS = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='LIPS')])

VBFA = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='VBFA')])

CDHDR = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='CDHDR')])
CDPOS = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='CDPOS')])

USR02 = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='USR02')])
MARA = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='MARA')])
KNA1 = pd.DataFrame(columns=[col[0] for col in sap_table.fetch_table(table_name='KNA1')])


In [3]:
class User:
    def __init__(self, bname, ustyp, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.bname = bname
        self.ustyp = ustyp

class Material:
    def __init__(self, matnr, price, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.matnr = matnr
        self.price = price

class Customer:
    def __init__(self, kunnr, erdat, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.kunnr = kunnr
        self.erdat = erdat
        
class SalesOrderItem:
    def __init__(self, vbeln, posnr, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.vbeln = vbeln
        self.posnr = posnr

In [4]:
# insert tables and create objects list
USERS = []
for k, v in values.users.items():
    users_last_index = len(USR02)
    USR02.loc[users_last_index, 'MANDT'] = values.mandt
    USR02.loc[users_last_index, 'BNAME'] = k
    USR02.loc[users_last_index, 'USTYP'] = v

    USERS.append(User(bname=k, ustyp=v))

CUSTOMERS = []
for k, v in values.customers.items():
    kna1_last_index = len(KNA1)
    rand_kunnr = uuid.uuid4()
    rand_erdat = helpers.generate_random_datetime(start_date=datetime(2009, 1, 1), end_date=datetime(2010, 1, 1))

    KNA1.loc[kna1_last_index, 'MANDT'] = values.mandt
    KNA1.loc[kna1_last_index, 'KUNNR'] = rand_kunnr

    CUSTOMERS.append(Customer(kunnr=rand_kunnr, erdat=rand_erdat))

MATERIALS = []
for k, v in values.materials.items():
    mara_last_index = len(MATERIALS)
    rand_matnr = uuid.uuid4()

    MARA.loc[mara_last_index, 'MANDT'] = values.mandt
    MARA.loc[mara_last_index, 'MATNR'] = rand_matnr

    MATERIALS.append(Material(matnr=rand_matnr, price=v['price']))

In [ ]:
class SalesOrder:
    def __init__(self, ernam, erdat, vbtyp) -> None:
        self.mandt = values.mandt
        self.vbeln = uuid.uuid4()
        self.erdat = erdat
        self.ernam = ernam
        self.vbtyp = vbtyp
        self.netwr = 0

        self.vbaps = []

        self.activity_create_sales_order()

    def activity_create_sales_order(self):
        vbak_last_index = len(VBAK)
        VBAK.loc[vbak_last_index, 'MANDT'] = self.mandt
        VBAK.loc[vbak_last_index, 'VBELN'] = self.vbeln
        VBAK.loc[vbak_last_index, 'ERNAM'] = self.ernam
        VBAK.loc[vbak_last_index, 'ERDAT'] = self.erdat
        VBAK.loc[vbak_last_index, 'VBTYP'] = self.vbtyp
    
    def activity_create_sales_order_item(self, materials: list):
        for i, mat in enumerate(materials):
            quantity = random.randint(25, 150)
            vbap_last_index = len(VBAP)

            VBAP.loc[vbap_last_index, 'MANDT'] = self.mandt
            VBAP.loc[vbap_last_index, 'VBELN'] = self.vbeln
            VBAP.loc[vbap_last_index, 'POSNR'] = i+1
            VBAP.loc[vbap_last_index, 'MATNR'] = mat.matnr
            VBAP.loc[vbap_last_index, 'KWMENG'] = quantity # Cumulative Order Quantity in Sales Units
            VBAP.loc[vbap_last_index, 'NETWR'] = mat.price * 12 * 12 * quantity # calculated as a box which usually contains 12 dozens
            VBAP.loc[vbap_last_index, 'VBELN'] = self.vbeln

            self.netwr += mat.price * 12 * 12 * quantity
            self.vbaps.append(SalesOrderItem(vbeln=self.vbeln, posnr=i))

    def activity_generate_delivery_doc(self, planned_delivery_at: datetime, activity_by: User):        
        # generate the delivery document
        rand_likp_id = uuid.uuid4()
        likp_last_index = len(LIKP)
        
        LIKP.loc[likp_last_index, 'MANDT'] = self.mandt
        LIKP.loc[likp_last_index, 'VBELN'] = rand_likp_id

        # record LIPS
        for vbap in self.vbaps:
            lips_last_index = len(LIPS)

            LIPS.loc[lips_last_index, 'MANDT'] = self.mandt
            LIPS.loc[lips_last_index, 'VBELN'] = rand_likp_id
            LIPS.loc[lips_last_index, 'POSNR'] = vbap.posnr

            # record the document flow
            vbfa_last_index = len(vbfa_last_index)
            VBFA.loc[vbfa_last_index, 'MANDT'] = self.mandt
            VBFA.loc[vbfa_last_index, 'VBELV'] = vbap.vbeln
            VBFA.loc[vbfa_last_index, 'POSNV'] = vbap.posnr
            VBFA.loc[vbfa_last_index, 'VBTYP_V'] = 'C'
            VBFA.loc[vbfa_last_index, 'VBELN'] = rand_likp_id
            VBFA.loc[vbfa_last_index, 'POSNN'] = vbap.posnr
            VBFA.loc[vbfa_last_index, 'VBTYP_N'] = 'J'

    # TODO
    def activity_release_delivery(self, delivery_released_at: datetime, activity_by: datetime):
      
        # record change
        self.cdhdr_changes_temp_list.append([values.mandt, 'DELIVERY', self.vbeln, rand_changenr, rand_usr, delivery_released_at])
        self.cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'LFSTK', pd.NA, 'DELIVERYRELEASED'])
        
        self.vbuk = [values.mandt, self.vbeln, 'DELIVERYRELEASED', pd.NA, delivery_released_at, 'C']



In [ ]:

    def release_delivery(self):

    def ship_goods(self):
        # Activity: Ship goods
        rand_mblnr = uuid.uuid4()
        goods_shipped_at = self.latest_activity_at + helpers.UPTO_WEEK()
        self.update_latest_activity_time(new_latest_time=goods_shipped_at)

        # assign user
        if random.uniform(0, 1) < 0.3: # 30% automation rate
            user = 'BATCH_JOB'
        else:
            user = random.choice(list(values.users))

        self.mkpf_temp_list.append([values.mandt, rand_mblnr, 'GOODSISSUE', goods_shipped_at, user])

        # cycle time
        cycle_time = (self.latest_activity_at - self.erdat).days
        if cycle_time >= 15:
            self.transition_matrix[self.tmi['ship_goods'], self.tmi['return_goods']] = 0.60
            self.transition_matrix[self.tmi['ship_goods'], self.tmi['send_invoice']] = 0.10
            self.transition_matrix[self.tmi['ship_goods'], self.tmi['cancel_order']] = 0.30
        
        for vbap_elem in self.vbaps:
            self.mseg_temp_list.append([values.mandt, rand_mblnr, self.vbeln, vbap_elem.posnr])

    def send_invoice(self):
        # assign user
        if random.uniform(0, 1) < 0.3: # 30% automation rate
            user = 'BATCH_JOB'
        else:
            user = random.choice(list(values.users))

        # impact of user type
        if user == 'BATCH_JOB':
            invoice_sent_at =  self.latest_activity_at + helpers.UPTO_HOUR()
        else:
            invoice_sent_at =  self.latest_activity_at + helpers.UPTO_DAY()
        self.update_latest_activity_time(new_latest_time=invoice_sent_at)

        # cycle time
        cycle_time = (self.latest_activity_at - self.erdat).days
        if cycle_time >= 15:
            self.transition_matrix[self.tmi['send_invoice'], self.tmi['return_goods']] = 0.60
            self.transition_matrix[self.tmi['send_invoice'], self.tmi['clear_invoice']] = 0.10
            self.transition_matrix[self.tmi['send_invoice'], self.tmi['cancel_order']] = 0.30

        rand_vbrk_id = uuid.uuid4()
        self.vbrk_temp_list.append([values.mandt, rand_vbrk_id, 'INVOICE', user, invoice_sent_at])
        
        self.send_invoice_items(vbrk_id=rand_vbrk_id)

    def send_invoice_items(self, vbrk_id: uuid.UUID):
        for vbap_elem in self.vbaps:
            self.vbrp_temp_list.append([values.mandt, vbrk_id, self.vbeln, vbap_elem.posnr])
    
    def receive_delivery_confirmation(self):
        # Activity: Recieve delivery confirmation
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users))

        delivery_confirmed_at = self.latest_activity_at + helpers.UPTO_WEEK()
        self.update_latest_activity_time(new_latest_time=delivery_confirmed_at)

        self.vbuk = [values.mandt, self.vbeln, 'DELIVERYCONFIRMED', pd.NA, delivery_confirmed_at, 'C']

        # record change
        self.cdhdr_changes_temp_list.append([values.mandt, 'DELIVERY', self.vbeln, rand_changenr, rand_usr, delivery_confirmed_at])
        self.cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'LFSTK', 'DELIVERYRELEASED', 'DELIVERYCONFIRMED'])

        # throughput time
        throughput_time = (self.latest_activity_at - self.erdat).days
        if throughput_time >= 20:
            self.transition_matrix[self.tmi['receive_delivery_confirmation'], self.tmi['return_goods']] = 0.60
            self.transition_matrix[self.tmi['receive_delivery_confirmation'], self.tmi['clear_invoice']] = 0.10
            self.transition_matrix[self.tmi['receive_delivery_confirmation'], self.tmi['cancel_order']] = 0.30

    def clear_invoice(self):
        # Activity: Clear invoice
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users))

        # DSO based on credit worthyness of the company
        invoice_cleared_at = self.latest_activity_at + timedelta(days=int(10/self.company.credit_worthyness))
        self.update_latest_activity_time(new_latest_time=invoice_cleared_at)

        self.vbuk = [values.mandt, self.vbeln, 'DELIVERYCONFIRMED', 'INVOICECLEARED', invoice_cleared_at, 'C']

        # record change
        self.cdhdr_changes_temp_list.append([values.mandt, 'BILLING', self.vbeln, rand_changenr, rand_usr, invoice_cleared_at])
        self.cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'FKSTK', pd.NA, 'INVOICECLEARED'])

class BillingDeviation:
    def __init__(self, vbak: SalesOrder) -> None:
        self.vbak = vbak
        
    def set_billing_block(self):
        # Activity: Set billing block
        self.vbak.faksk = 'BILLINGBLOCK'

        rand_changenr = uuid.uuid4()

        # assign user
        if random.uniform(0, 1) < 0.6: 
            user = 'BATCH_JOB'
        else:
            user = random.choice(list(values.users))

        # impact of user type
        if user == 'BATCH_JOB':
            billing_block_set_at =  self.vbak.latest_activity_at + helpers.UPTO_HOUR()
        else:
            billing_block_set_at =  self.vbak.latest_activity_at + helpers.UPTO_DAY()

        # increase cycle time
        self.vbak.update_latest_activity_time(new_latest_time=billing_block_set_at + helpers.UPTO_MONTH())

        self.vbak.cdhdr_changes_temp_list.append([values.mandt, 'BILLING', self.vbak.vbeln, rand_changenr, user, billing_block_set_at])
        self.vbak.cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBAK', 'FAKSK', pd.NA, 'BILLINGBLOCK'])
        
        self.vbak.has_billing_block = True

    def remove_billing_block(self):
        # Activity: Remove billing block
        self.vbak.faksk = pd.NA

        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users))

        billing_block_removed_at = self.vbak.latest_activity_at + helpers.UPTO_WEEK()
        self.vbak.update_latest_activity_time(new_latest_time=billing_block_removed_at)

        self.vbak.cdhdr_changes_temp_list.append([values.mandt, 'BILLING', self.vbak.vbeln, rand_changenr, rand_usr, billing_block_removed_at])
        self.vbak.cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBAK', 'FAKSK', 'BILLINGBLOCK', pd.NA])

class CancelOrReturn:
    def __init__(self, vbak) -> None:
        self.vbak = vbak
    
    def cancel_order(self):
        # Activity: Cancel order
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users))

        sd_cancelled_at = self.vbak.latest_activity_at + helpers.UPTO_DAY()
        self.vbak.update_latest_activity_time(new_latest_time=sd_cancelled_at)

        self.vbak.cdhdr_changes_temp_list.append([values.mandt, 'SALESORDER', self.vbak.vbeln, rand_changenr, rand_usr, sd_cancelled_at])
        self.vbak.cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'VBTYP', self.vbak.vbuk[-1], 'h'])
        self.vbak.vbuk[-1] = 'h'

    def return_goods(self):
        # Activity: Return goods
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users))

        sd_returned_at = self.vbak.latest_activity_at + helpers.UPTO_MONTH()
        self.vbak.update_latest_activity_time(new_latest_time=sd_returned_at)

        self.vbak.cdhdr_changes_temp_list.append([values.mandt, 'SALESORDER', self.vbak.vbeln, rand_changenr, rand_usr, sd_returned_at])
        self.vbak.cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'VBTYP', self.vbak.vbuk[-1], 'H'])
        self.vbak.vbuk[-1] = 'H'